In [19]:
import pandas as pd
import re

#df : 스크래핑한 호주 문서
df = pd.read_csv("호주_세관_최근10.csv")

df_title = list(df['title'])
df_text = list(df['text'])
df_add = []

for i in range(len(df)):
    df_add.append(df_title[i]+""+df_text[i])

print(len(df_add))

#df_keyword : 선정한 호주 키워드 101개
df_keyword = pd.read_csv("호주_번역_100.csv", index_col = False)
print(len(df_keyword))

10
102


In [20]:
import numpy as np
import itertools
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# SentenceTransformer : 모델 훈련 위한 라이브러리
# 'distiluse-base-multilingual-cased-v1' : 사용할 모델
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [21]:
#keyword : 불러온 호주 키워드 중 번역 Column 추출
keyword = list(df_keyword["번역"])
print(len(keyword))

102


In [22]:
#키워드 벡터화
keyword_embedding = []

for ele in keyword:
    keyword_embedding.append(model.encode(ele))

print(len(keyword_embedding))

102


In [23]:
#호주 문서 10개 embedding
import nltk
import string
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

doc_embedding = []
doc_words = []
cnt = 1

for doc in df_add:
    result = []
    sentence = ''
    
#    #1. 전처리 전 문자 길이
#      print(f"{cnt}번 문서 전처리 전 문자 길이 : {len(doc)}")
    
#    #2. 공백 제거 + strip()
    doc = doc.replace("\n", "").strip()
#     print(f"{cnt}번 문서 공백 제거 후 문자 길이 : {len(doc)}")
    
#     #3. 구두점 제거
#     doc = "".join([i for i in doc if i not in string.punctuation]).strip()
#     print(f"{cnt}번 문서 구두점 제거 후 문자 길이 : {len(doc)}")
    
#     print(f"{cnt}번 문서 불용어 제거 후 문자 길이 : {len(doc)}")
    
    #5. 문서 토큰화
    word_tokens = pos_tag(word_tokenize(doc))

    word_tokens = [t[0] for t in word_tokens if t[1] == "NN" and len(t[0]) > 2]
    for w in word_tokens:  
        if w not in stop_words:
            result.append(w)
    result = [word.lower() for word in result]
    doc_words.append(result)
    doc_embedding.append(model.encode(result))
    
    #카운트 추가
    print(cnt, "번 문서 진행 중")
    cnt += 1 

1 번 문서 진행 중
2 번 문서 진행 중
3 번 문서 진행 중
4 번 문서 진행 중
5 번 문서 진행 중
6 번 문서 진행 중
7 번 문서 진행 중
8 번 문서 진행 중
9 번 문서 진행 중
10 번 문서 진행 중


In [24]:
#키워드와 문서 유사도 비교 0.9
doc_keywords_9 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.9:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_9.append(set(final_result))

set()
{'car'}
set()
set()
set()
set()
set()
set()
{'plastic'}
set()


In [25]:
#키워드와 문서 유사도 비교 0.8
doc_keywords_8 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.8:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_8.append(set(final_result))

set()
{'measure', 'car'}
set()
set()
set()
set()
set()
set()
{'plastic', 'measure'}
set()


In [18]:
#키워드와 문서 유사도 비교 0.75
doc_keywords_75 = []
for index, doc in enumerate(doc_embedding):
    final_result = []
    for i in range(len(doc)):
        for j in range(len(keyword_embedding)):
            distances = cosine_similarity([doc[i]], [keyword_embedding[j]])
            if distances[0][0]>0.75:
                final_result.append(doc_words[index][i])
# distances = cosine_similarity([doc_embedding[0]], candidate_embeddings)
    print(set(final_result))
    doc_keywords_75.append(set(final_result))

{'fuel', 'beers', 'occurring'}
{'belarus', 'fuel', 'vehicles', 'measure', 'directed', 'car'}
{'make', 'molonglo'}
{'urp', 'belarus'}
{'nsw', 'hageman', 'make', 'brien', 'molonglo', 'wang', 'plummer', 'gatt', 'bradley'}
set()
{'fisheries', 'latha', 'agriculture'}
{'medicaments', 'medical'}
{'plastics', 'baled', 'stamatis', 'fuel', 'plastic', 'measure'}
{'make', 'molonglo'}


In [28]:
df['cosine 0.9'] = doc_keywords_9
df['cosine 0.8'] = doc_keywords_8
df['cosine 0.75'] = doc_keywords_75

df

,date,link,title,text,cosine 0.9,cosine 0.8,cosine 0.75
0,2022-07-28,https://www.abf.gov.au/help-and-support-subsit...,Indexation of customs duty rates on excise-equ...,Australian Customs Notice \nNo. 2022/35 \nInd...,{},{},"{fuel, beers, occurring}"
1,2022-07-22,https://www.abf.gov.au/help-and-support-subsit...,Removal of customs duty on certain electric ve...,Australian Customs Notice \nNo. 2022/34 \nRem...,{car},"{measure, car}","{belarus, fuel, vehicles, measure, directed, car}"
2,2022-07-21,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/33 \n\nAppl...,{},{},"{make, molonglo}"
3,2022-07-12,https://www.abf.gov.au/help-and-support-subsit...,Temporary duty reduction for goods from Ukraine,Australian Customs Notice \nNo. 2022/32 \nTem...,{},{},"{urp, belarus}"
4,2022-07-11,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/31 \n\nAppl...,{},{},"{nsw, hageman, make, brien, molonglo, wang, pl..."
5,2022-07-01,https://www.abf.gov.au/help-and-support-subsit...,Cargo reporting requirements for international...,Australian Customs Notice (ACN) \n2022/30 \nC...,{},{},{}
6,2022-07-01,https://www.abf.gov.au/help-and-support-subsit...,New timeframes for the disposal of unentered a...,Australian Customs Notice (ACN) \n2022/29 \n\n...,{},{},"{fisheries, latha, agriculture}"
7,2022-06-29,https://www.abf.gov.au/help-and-support-subsit...,Tariff Concession – Item 57 – Revised,Australian Customs Notice \nNo. 2022/28 \nTar...,{},{},"{medicaments, medical}"
8,2022-06-28,https://www.abf.gov.au/help-and-support-subsit...,Commencement of phase 2 of the Recycling and W...,Australian Customs Notice \nNo. 2022/27 \nCom...,{plastic},"{plastic, measure}","{plastics, baled, stamatis, fuel, plastic, mea..."
9,2022-06-27,https://www.abf.gov.au/help-and-support-subsit...,Application for Customs Broker Licences,Australian Customs Notice No. 2022/26 \n\nAppl...,{},{},"{make, molonglo}"


In [30]:
df.to_csv("호주_세관_최근10_유사도, 번역 추가.csv")